In [ ]:
!pip install gradio -q
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import model_from_json
import gradio as gr
from keras.models import load_model

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 9.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.6/314.6 kB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 50.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 7.7 MB/s et

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import model_from_json

def predict_prices(start_date, n_days):
    # Load model architecture from JSON file
    with open("model.json", "r") as json_file:
        loaded_model_json = json_file.read()
    # Load the model
    model = model_from_json(loaded_model_json)
    # Load model weights
    model.load_weights("model.h5")

    # Load your preprocessed data
    df = pd.read_csv('/content/BTC-USD.csv')
    df['Date'] = pd.to_datetime(df['Date'])
    df.dropna(inplace=True)
    df['Volume'] = pd.to_numeric(df['Volume'], errors='coerce')
    # Assuming you've already defined look_back
    look_back = 15
    # Preprocess the start date
    start_date = pd.to_datetime(start_date)
    # Find the index corresponding to the provided start date
    start_index = df[df['Date'] == start_date].index.tolist()[0]
    # Extract the relevant portion of the data for prediction
    data = df['Close'].values[start_index-look_back:start_index].reshape(-1, 1)
    # Scale the data
    min_max_scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = min_max_scaler.fit_transform(data)
    # Prepare input sequences for prediction
    x_input = scaled_data.reshape(1, -1)
    temp_input = list(x_input[0])
    lst_output = []
    n = n_days  # next number of days for which we are predicting
    i = 0
    predicted_prices = []
    while(i < n):
        if(len(temp_input) > look_back):
            x_input = np.array(temp_input[1:])
            x_input = x_input.reshape(1, -1)
            x_input = x_input.reshape((1, look_back, 1))
            yhat = model.predict(x_input, verbose=0)
            temp_input.extend(yhat[0].tolist())
            temp_input = temp_input[1:]
            lst_output.extend(yhat.tolist())
            predicted_price = min_max_scaler.inverse_transform(np.array(yhat).reshape(-1, 1))[0][0]
            predicted_prices.append(f"Predicted price for Day {i+1} is {predicted_price:.2f}")
            i += 1
        else:
            x_input = x_input.reshape((1, look_back, 1))
            yhat = model.predict(x_input, verbose=0)
            temp_input.extend(yhat[0].tolist())
            lst_output.extend(yhat.tolist())
            predicted_price = min_max_scaler.inverse_transform(np.array(yhat).reshape(-1, 1))[0][0]
            predicted_prices.append(f"Predicted price for Day {i+1} is {predicted_price:.2f}")
            i += 1
    return predicted_prices


In [ ]:
date_input = gr.Textbox(label = "Enter Starting date")
days_input = gr.Number(label = "Enter the number of days")
output = gr.Textbox(label="Predicted prices")

In [ ]:
# Create Gradio interface
interface = gr.Interface(
    fn=predict_prices,
    inputs=[date_input, days_input],
    outputs=output,
    title="Bitcoin Price Prediction",
    description="Enter the starting date and the number of days to predict Bitcoin prices."
)

# Launch the Gradio interface
interface.launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://499eb855e942e09476.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
